The playlist followed

https://www.youtube.com/playlist?list=PLQY2H8rRoyvzDbLUZkbudP-MFQZwNmU4S

Libraries

In [31]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json
import tensorflow as tf
import numpy as np

Tokenization

In [3]:
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://github.com/keras-team/keras/blob/v3.3.3/keras/src/legacy/preprocessing/text.py#L81-L312

sentences = [
    "I love my dog",
    "I love my cat",
    "We love our dog!"
]

tokenizer = Tokenizer(num_words=100) # max words is the i/p parameter.
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(word_index) # prints a dictionary of word and its label
# Its possibly prioritizing words that occur frequently

{'love': 1, 'i': 2, 'my': 3, 'dog': 4, 'cat': 5, 'we': 6, 'our': 7}


Sequencing

In [4]:
# Creates sequence of integers (labels) for the text provided. 
# The labels (word_index) is obtained from tokernizer.

sentences = [
    "I love my dog",
    "I love my cat",
    "We love our dog!",
    "Do you think our dog is amazing?"
]

# To handle words that are not used for training, a oov_token value can be given,
# this at least preserves the sentence length although some information is lost.
tokenizer = Tokenizer(num_words=100, oov_token="<oov>") # max words is the i/p parameter.
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(sentences)
print(word_index)
print(sequences)

test_data = [
    "I really love my dog",
    "My dog loves its food"
]

test_seq = tokenizer.texts_to_sequences(test_data)
print(test_seq) # note the word index of each word in the sentence

# Inorder to handle words of different lengths for training NNs., we use padding.
padded = pad_sequences(sequences, padding="post") # post - adds zeros at the end, 
# note that sentence size is made uniform refering the longest sentence provided,
# max_len param can be used to set max sentence size.
# But if sentence length is longer than max_len, then a truncation step can be used using 'truncating' parameter, set as 'pre' or 'post'.

print(padded)

{'<oov>': 1, 'love': 2, 'dog': 3, 'i': 4, 'my': 5, 'our': 6, 'cat': 7, 'we': 8, 'do': 9, 'you': 10, 'think': 11, 'is': 12, 'amazing': 13}
[[4, 2, 5, 3], [4, 2, 5, 7], [8, 2, 6, 3], [9, 10, 11, 6, 3, 12, 13]]
[[4, 1, 2, 5, 3], [5, 3, 1, 1, 1]]
[[ 4  2  5  3  0  0  0]
 [ 4  2  5  7  0  0  0]
 [ 8  2  6  3  0  0  0]
 [ 9 10 11  6  3 12 13]]


Train model for sentiment analysis

In [ ]:
# Data re-formating
with open("sarcasm.json", 'r') as f: # File obtained from https://storage.googleapis.com/learning-datasets/sarcasm.json
    datastore = json.load(f)

sentences = []
labels = []
urls = []
for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])
    urls.append(item['article_link'])

# Get tokens for each word with a oov token set to deal with unknown words
tokenizer = Tokenizer(oov_token="<oov>")
tokenizer.fit_on_texts(sentences)
# Get word index
word_index = tokenizer.word_index
# Generate sequence of word index 
sequences = tokenizer.texts_to_sequences(sentences)
# Add padding to maintain sentence length
padded = pad_sequences(sequences, padding="post")
print(padded[0])
print(padded.shape)

[  308 15115   679  3337  2298    48   382  2576 15116     6  2577  8434
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]
(26709, 40)


Train-Test Split

In [24]:
# Splitting the data into training and test set as we normally will not be useful,
# as we have to ensure the neural network sees only the training data.
# Here's how we can re-write the train-test split

vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences) # fitting only on training sequences.
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


Embedding

In [25]:
# Getting meaning of words used in sentences i.e their significance.
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 24)                408       
                                                                 
 dense_3 (Dense)             (None, 1)                 25        
                                                                 
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


Training

In [32]:
num_epochs = 30
training_labels = np.array(training_labels)  # Convert to NumPy array
testing_labels = np.array(testing_labels)  # Convert to NumPy array


history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/30
625/625 - 1s - loss: 0.6797 - accuracy: 0.5619 - val_loss: 0.6482 - val_accuracy: 0.5906 - 1s/epoch - 2ms/step
Epoch 2/30
625/625 - 1s - loss: 0.4722 - accuracy: 0.8030 - val_loss: 0.3900 - val_accuracy: 0.8362 - 801ms/epoch - 1ms/step
Epoch 3/30
625/625 - 1s - loss: 0.3188 - accuracy: 0.8732 - val_loss: 0.3573 - val_accuracy: 0.8477 - 750ms/epoch - 1ms/step
Epoch 4/30
625/625 - 1s - loss: 0.2666 - accuracy: 0.8943 - val_loss: 0.3454 - val_accuracy: 0.8541 - 866ms/epoch - 1ms/step
Epoch 5/30
625/625 - 1s - loss: 0.2303 - accuracy: 0.9109 - val_loss: 0.3445 - val_accuracy: 0.8541 - 835ms/epoch - 1ms/step
Epoch 6/30
625/625 - 1s - loss: 0.2039 - accuracy: 0.9211 - val_loss: 0.3509 - val_accuracy: 0.8566 - 897ms/epoch - 1ms/step
Epoch 7/30
625/625 - 1s - loss: 0.1815 - accuracy: 0.9305 - val_loss: 0.3674 - val_accuracy: 0.8527 - 919ms/epoch - 1ms/step
Epoch 8/30
625/625 - 1s - loss: 0.1639 - accuracy: 0.9391 - val_loss: 0.3930 - val_accuracy: 0.8432 - 881ms/epoch - 1ms/step
Epo